In [1]:
import pandas as pd
from silhouette_upper_bound import upper_bound, upper_bound_samples, upper_bound_macro_silhouette
import numpy as np
import matplotlib.pyplot as plt 
import pickle
from sklearn.metrics import silhouette_score, adjusted_rand_score, silhouette_samples, adjusted_mutual_info_score
from collections import Counter
import kmedoids
from sklearn.preprocessing import StandardScaler, RobustScaler, normalize
from sklearn.impute import SimpleImputer
from scipy.spatial.distance import squareform, pdist
import seaborn as sns
from matplotlib.ticker import MultipleLocator
from tqdm import tqdm
from pathlib import Path
from scipy.io import arff
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
import time

In [2]:
# Distance metric = L1

In [3]:
dataset_name = "aloi-hsb-5x5x5"
path = "data/aloi/" + dataset_name + ".csv"
N_FEATURES = 125

In [4]:
# Load whitespace-separated data
df = pd.read_csv(path, 
                 sep=r"\s+",      # whitespace
                 header=None,     # no header row
                 engine="python") # needed because of regex separator

print(df.shape)
df.head()

(110250, 127)


,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,0.006198,0.000000,0.0,0.0,0.000002,0.093140,0.000054,0.000005,0.000000,0.002606,...,0.000057,0.000052,0.000217,0.000294,0.001180,0.002249,0.010247,0.016556,"""img1""","""1/1_i110.png"""
1,0.007600,0.000000,0.0,0.0,0.000005,0.099254,0.000104,0.000011,0.000002,0.006027,...,0.000047,0.000072,0.000271,0.000294,0.000875,0.002636,0.011791,0.012607,"""img1""","""1/1_i120.png"""
2,0.009555,0.000002,0.0,0.0,0.000050,0.113557,0.000212,0.000072,0.000029,0.013439,...,0.000059,0.000075,0.000328,0.000217,0.000882,0.002471,0.011339,0.011084,"""img1""","""1/1_i130.png"""
3,0.013756,0.000016,0.0,0.0,0.007103,0.140993,0.000382,0.000237,0.000179,0.022142,...,0.000066,0.000061,0.000545,0.000176,0.001205,0.001029,0.004071,0.022572,"""img1""","""1/1_i140.png"""
4,0.014931,0.000016,0.0,0.0,0.000301,0.130830,0.000723,0.000744,0.001067,0.020598,...,0.000070,0.000120,0.000373,0.000179,0.000626,0.003092,0.012356,0.005875,"""img1""","""1/1_i150.png"""


In [5]:
df["class"] = df.iloc[:, -1].str.extract(r"(\d+)", expand=False)
df = df[df["class"].notna()]   # drop rows with missing class
df["class"] = df["class"].astype(int)

In [6]:
df_subset = (
    df.groupby("class", group_keys=False)
      .apply(lambda g: g.sample(n=40, random_state=42))
      .reset_index(drop=True)
)

/var/folders/40/l50vx_fn2nd9hv4y_ssvkhn80000gn/T/ipykernel_50476/3005663608.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=40, random_state=42))


In [7]:
df_subset

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,class
0,0.045401,0.001526,0.003160,0.006736,0.023887,0.172085,0.004343,0.011235,0.008873,0.003832,...,0.000411,0.000066,0.000335,0.002301,0.009011,0.006079,0.000045,"""img1""","""1/1_r280.png""",1
1,0.045225,0.000613,0.001031,0.002921,0.020752,0.173089,0.002421,0.004028,0.005229,0.005391,...,0.000271,0.000416,0.000201,0.000520,0.004795,0.011194,0.002145,"""img1""","""1/1_i230.png""",1
2,0.014931,0.000016,0.000000,0.000000,0.000301,0.130830,0.000723,0.000744,0.001067,0.020598,...,0.000120,0.000373,0.000179,0.000626,0.003092,0.012356,0.005875,"""img1""","""1/1_i150.png""",1
3,0.043287,0.001560,0.003798,0.005082,0.019194,0.170754,0.006427,0.012146,0.007935,0.003599,...,0.000330,0.000183,0.000183,0.001350,0.007388,0.009465,0.000136,"""img1""","""1/1_r305.png""",1
4,0.050345,0.001476,0.003187,0.005841,0.028332,0.177456,0.003007,0.005674,0.006933,0.004853,...,0.000543,0.000079,0.000104,0.002783,0.017675,0.003879,0.000070,"""img1""","""1/1_r210.png""",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.023562,0.000194,0.000095,0.000084,0.000384,0.161662,0.000726,0.001921,0.001548,0.001279,...,0.001094,0.002498,0.000066,0.003556,0.011974,0.006391,0.002857,"""img1000""","""1000/1000_r230.png""",1000
39996,0.031325,0.000023,0.000038,0.000147,0.004896,0.156569,0.000226,0.000086,0.000269,0.003108,...,0.000208,0.003513,0.000269,0.002699,0.004338,0.004820,0.008850,"""img1000""","""1000/1000_l5c3.png""",1000
39997,0.022185,0.000090,0.000138,0.000052,0.000104,0.160437,0.001022,0.001940,0.001535,0.000305,...,0.000484,0.001126,0.000109,0.006192,0.008739,0.002066,0.002787,"""img1000""","""1000/1000_r10.png""",1000
39998,0.014642,0.000009,0.000000,0.000000,0.000133,0.140951,0.000160,0.000081,0.000222,0.000637,...,0.000305,0.001273,0.000032,0.003897,0.015365,0.011133,0.004892,"""img1000""","""1000/1000_i150.png""",1000


In [8]:
y = df_subset["class"]
X = df_subset.iloc[:, 0:N_FEATURES]
X = X.to_numpy(dtype='float32')
print(X.shape, X.dtype)

(40000, 504) float32


In [9]:
D = np.load(f"arrays/aloi-{dataset_name}-D.npy")

In [10]:
print(D.shape, D.dtype)

(40000, 40000) float32


In [11]:
start = time.perf_counter()
ub = upper_bound(D) 
print(f"RT = {time.perf_counter() - start}")

RT = 229.25150637499996


In [12]:
ub

np.float64(0.9005815737721546)

### dynmsc for K in range 2 to 15

In [13]:
start = time.perf_counter()
cluster_labels = (kmedoids.dynmsc(diss=D, medoids=15, random_state=42).labels + 1) 
print(f"RT = {time.perf_counter() - start}")

RT = 554.65869075


In [14]:
cluster_sizes = list(Counter(cluster_labels).values())
min_cluster_size = min(cluster_sizes)
print(f"Min cluster size = {min_cluster_size}")
print(f"K = {len(cluster_sizes)}")
# silhouette samples 
silh_samples = silhouette_samples(X=D, labels=cluster_labels, metric='precomputed')
# ASW 
asw = np.mean(silh_samples)
print(f"ASW = {asw}")
print(f"ub = {ub}")
print(f"WCRE = {(ub - asw)/ub}")

Min cluster size = 38
K = 2
ASW = 0.4702458381652832
ub = 0.9005815737721546
WCRE = 0.4778420391218725


In [15]:
start = time.perf_counter()
uba = upper_bound(D, min_cluster_size) 
print(f"RT = {time.perf_counter() - start}")
print(f"uba = {uba}")
print(f"WCRE = {(uba - asw)/uba}")

RT = 221.87945583400005
uba = 0.751921238653854
WCRE = 0.374607586551016


In [16]:
# AMI and ARI 
ari = adjusted_rand_score(cluster_labels, y)
ami = adjusted_mutual_info_score(cluster_labels, y)

print(f"Adjusted Rand Index vs. true labels: {ari:.3f}")
print(f"Adjusted Mutual Info vs. true labels: {ami:.3f}")

Adjusted Rand Index vs. true labels: 0.000
Adjusted Mutual Info vs. true labels: 0.001


### fastmsc for K = 1000

In [17]:
start = time.perf_counter()
cluster_labels = (kmedoids.fastmsc(diss=D, medoids=1000, random_state=42).labels + 1) 
print(f"RT = {time.perf_counter() - start}")

RT = 1075.0374202079997


In [18]:
cluster_sizes = list(Counter(cluster_labels).values())
min_cluster_size = min(cluster_sizes)
print(f"Min cluster size = {min_cluster_size}")
print(f"K = {len(cluster_sizes)}")
# silhouette samples 
silh_samples = silhouette_samples(X=D, labels=cluster_labels, metric='precomputed')
# ASW 
asw = np.mean(silh_samples)
print(f"ASW = {asw}")
print(f"ub = {ub}")
print(f"WCRE = {(ub - asw)/ub}")

Min cluster size = 2
K = 1000
ASW = 0.14187544584274292
ub = 0.9005815737721546
WCRE = 0.8424624154273035


In [19]:
start = time.perf_counter()
uba = upper_bound(D, min_cluster_size) 
print(f"RT = {time.perf_counter() - start}")
print(f"uba = {uba}")
print(f"WCRE = {(uba - asw)/uba}")

RT = 257.9209444580001
uba = 0.9004619498705162
WCRE = 0.8424414869910448


In [20]:
# AMI and ARI 
ari = adjusted_rand_score(cluster_labels, y)
ami = adjusted_mutual_info_score(cluster_labels, y)

print(f"Adjusted Rand Index vs. true labels: {ari:.3f}")
print(f"Adjusted Mutual Info vs. true labels: {ami:.3f}")

Adjusted Rand Index vs. true labels: 0.292
Adjusted Mutual Info vs. true labels: 0.589


### n classes = 2

In [24]:
np.random.seed(872)
classes = df_subset["class"].unique()
n_classes = 2
chosen = np.random.choice(classes, size=n_classes, replace=False)
# Filter dataframe to those two classes
df_subset_2 = df_subset[df_subset["class"].isin(chosen)]

df_subset_2

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,class
6160,0.074300,0.003447,0.008617,0.016328,0.054172,0.210296,0.010435,0.020164,0.018543,0.013331,...,0.000000,0.000000,0.000018,0.000000,0.000000,0.0,0.0,"""img155""","""155/155_r280.png""",155
6161,0.094928,0.003348,0.006045,0.015627,0.072729,0.214281,0.010220,0.012153,0.016199,0.014633,...,0.000000,0.000000,0.000041,0.000002,0.000000,0.0,0.0,"""img155""","""155/155_i230.png""",155
6162,0.034772,0.000430,0.000264,0.000156,0.021170,0.203340,0.006698,0.008800,0.017675,0.065780,...,0.000000,0.000000,0.000014,0.000000,0.000000,0.0,0.0,"""img155""","""155/155_i150.png""",155
6163,0.074510,0.003493,0.008579,0.017877,0.049793,0.209434,0.011750,0.021550,0.017815,0.014504,...,0.000000,0.000000,0.000005,0.000002,0.000000,0.0,0.0,"""img155""","""155/155_r305.png""",155
6164,0.077076,0.003251,0.007751,0.018351,0.058393,0.211381,0.011974,0.018643,0.017013,0.012295,...,0.000000,0.000000,0.000018,0.000007,0.000000,0.0,0.0,"""img155""","""155/155_r210.png""",155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,0.067833,0.000034,0.000025,0.000005,0.000427,0.226483,0.000420,0.000113,0.000054,0.000170,...,0.000000,0.000000,0.000075,0.000000,0.000000,0.0,0.0,"""img389""","""389/389_r230.png""",389
15556,0.096117,0.000061,0.000014,0.000007,0.000714,0.217957,0.000834,0.000337,0.000061,0.000436,...,0.000057,0.000002,0.000014,0.000002,0.000005,0.0,0.0,"""img389""","""389/389_l5c3.png""",389
15557,0.066433,0.000034,0.000016,0.000007,0.000362,0.222224,0.000552,0.000057,0.000029,0.000206,...,0.000002,0.000000,0.000007,0.000000,0.000000,0.0,0.0,"""img389""","""389/389_r10.png""",389
15558,0.023388,0.000014,0.000002,0.000000,0.000314,0.192726,0.000167,0.000090,0.000020,0.000183,...,0.000002,0.000005,0.000018,0.000027,0.000005,0.0,0.0,"""img389""","""389/389_i150.png""",389


In [25]:
y2 = df_subset_2["class"]
X2 = df_subset_2.iloc[:, 0:N_FEATURES]
X2 = X2.to_numpy()
np.save("arrays/aloi125-2classes.npy", X2)
print(X2.shape, X2.dtype)
D2 = squareform(pdist(X2, metric="cityblock"))
ub = upper_bound(D2)
cluster_labels = (kmedoids.pamsil(diss=D2, medoids=n_classes, random_state=42).labels + 1) 

(80, 125) float64


In [26]:
cluster_sizes = list(Counter(cluster_labels).values())
min_cluster_size = min(cluster_sizes)
print(f"Min cluster size = {min_cluster_size}")
print(f"K = {len(cluster_sizes)}")
# silhouette samples 
silh_samples = silhouette_samples(X=D2, labels=cluster_labels, metric='precomputed')
# ASW 
asw = np.mean(silh_samples)
print(f"ASW = {asw}")
print(f"ub = {ub}")
print(f"WCRE = {(ub - asw)/ub}")

Min cluster size = 37
K = 2
ASW = 0.6754811511998335
ub = 0.8686601785202581
WCRE = 0.222387340984711


In [27]:
uba = upper_bound(D2, min_cluster_size)
print(f"uba = {uba}\nwcre = {(uba - asw)/uba}")

uba = 0.7194512265260196
wcre = 0.06111613088562273


In [11]:
# AMI and ARI 
ari = adjusted_rand_score(cluster_labels, y2)
ami = adjusted_mutual_info_score(cluster_labels, y2)

print(f"Adjusted Rand Index vs. true labels: {ari:.3f}")
print(f"Adjusted Mutual Info vs. true labels: {ami:.3f}")

Adjusted Rand Index vs. true labels: 0.854
Adjusted Mutual Info vs. true labels: 0.804


### n classes = 5

In [28]:
np.random.seed(25)
classes = df_subset["class"].unique()
n_classes = 5
chosen = np.random.choice(classes, size=n_classes, replace=False)
# Filter dataframe to those two classes
df_subset_2 = df_subset[df_subset["class"].isin(chosen)]

df_subset_2

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,class
1960,0.060918,0.000000,0.000000,0.000000,0.000000,0.205641,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000005,0.000000,0.000000,0.000000,0.000000,"""img50""","""50/50_r280.png""",50
1961,0.066673,0.000000,0.000000,0.000000,0.000000,0.214615,0.000007,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000002,0.000000,0.000000,0.000000,0.000000,"""img50""","""50/50_i230.png""",50
1962,0.022400,0.000000,0.000000,0.000000,0.000000,0.164062,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"""img50""","""50/50_i150.png""",50
1963,0.062414,0.000000,0.000000,0.000000,0.000000,0.206516,0.000014,0.000002,0.000000,0.000000,...,0.000000,0.000000,0.000007,0.000000,0.000000,0.000000,0.000000,"""img50""","""50/50_r305.png""",50
1964,0.060714,0.000000,0.000000,0.000000,0.000000,0.203218,0.000014,0.000000,0.000000,0.000000,...,0.000023,0.000000,0.000034,0.000000,0.000000,0.000000,0.000000,"""img50""","""50/50_r210.png""",50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31035,0.033090,0.000070,0.000129,0.000477,0.008755,0.154414,0.001200,0.001020,0.000977,0.001087,...,0.000154,0.000316,0.000129,0.001528,0.006619,0.011723,0.002331,"""img776""","""776/776_r230.png""",776
31036,0.045062,0.000122,0.000118,0.000206,0.018324,0.159991,0.000696,0.000364,0.000271,0.000540,...,0.000319,0.002150,0.000131,0.000886,0.003176,0.010245,0.024841,"""img776""","""776/776_l5c3.png""",776
31037,0.038407,0.000255,0.000934,0.001203,0.013735,0.160857,0.001001,0.001406,0.001155,0.001155,...,0.000165,0.000997,0.000158,0.005419,0.013145,0.012689,0.000906,"""img776""","""776/776_r10.png""",776
31038,0.016534,0.000052,0.000038,0.000057,0.010934,0.132930,0.000443,0.000522,0.000995,0.004259,...,0.000120,0.000475,0.000192,0.002215,0.007926,0.011536,0.013708,"""img776""","""776/776_i150.png""",776


In [29]:
y2 = df_subset_2["class"]
X2 = df_subset_2.iloc[:, 0:N_FEATURES]
X2 = X2.to_numpy()
np.save("arrays/aloi125-5classes.npy", X2)
print(X2.shape, X2.dtype)
D2 = squareform(pdist(X2, metric="cityblock"))
ub = upper_bound(D2)
cluster_labels = (kmedoids.pamsil(diss=D2, medoids=n_classes, random_state=42).labels + 1) 

(200, 125) float64


In [30]:
cluster_sizes = list(Counter(cluster_labels).values())
min_cluster_size = min(cluster_sizes)
print(f"Min cluster size = {min_cluster_size}")
print(f"K = {len(cluster_sizes)}")
# silhouette samples 
silh_samples = silhouette_samples(X=D2, labels=cluster_labels, metric='precomputed')
# ASW 
asw = np.mean(silh_samples)
print(f"ASW = {asw}")
print(f"ub = {ub}")
print(f"WCRE = {(ub - asw)/ub}")
print(cluster_sizes)

Min cluster size = 9
K = 5
ASW = 0.47127628674975325
ub = 0.8571444368288681
WCRE = 0.4501786787611792
[79, 9, 37, 41, 34]


In [31]:
uba = upper_bound(D2, min_cluster_size)
print(f"uba = {uba}\nwcre = {(uba - asw)/uba}")

uba = 0.7593424970766962
wcre = 0.3793626873722139


In [19]:
# AMI and ARI 
ari = adjusted_rand_score(cluster_labels, y2)
ami = adjusted_mutual_info_score(cluster_labels, y2)

print(f"Adjusted Rand Index vs. true labels: {ari:.3f}")
print(f"Adjusted Mutual Info vs. true labels: {ami:.3f}")

Adjusted Rand Index vs. true labels: 0.656
Adjusted Mutual Info vs. true labels: 0.747


In [14]:
def save_results(path, diss_matrix, k_range = range(2, 16), random_state = 42):

    if Path(path).exists():
        print("Path exists. Aborting.")
        return None 

    results = {}

    list_n_clusters = []
    list_cluster_labels = []
    list_cluster_sizes = []
    list_min_cluster_size = []
    list_silh_samples = []
    list_asw = []
    list_ub_asw = []
    list_ub_asw_min_cluster_size = []

    for k in tqdm(k_range):

        # cluster labels
        cluster_labels = (kmedoids.pamsil(diss=diss_matrix, medoids=k, random_state=random_state).labels + 1)

        # cluster sizes
        cluster_sizes = list(Counter(cluster_labels).values())
        min_cluster_size = min(cluster_sizes)

        # silhouette samples 
        silh_samples = silhouette_samples(X=diss_matrix, labels=cluster_labels, metric='precomputed')

        # ASW 
        asw = np.mean(silh_samples)
        ub_asw = upper_bound(diss_matrix)
        ub_asw_min_cluster_size = upper_bound(diss_matrix, m=min_cluster_size)

        print(ub_asw)

        list_n_clusters.append(k)
        list_cluster_labels.append(cluster_labels)
        list_cluster_sizes.append(cluster_sizes)
        list_min_cluster_size.append(min_cluster_size)
        list_silh_samples.append(silh_samples)
        list_asw.append(asw)
        list_ub_asw.append(ub_asw)
        list_ub_asw_min_cluster_size.append(ub_asw_min_cluster_size)
    
    results = {
        "n_clusters": list_n_clusters,
        "cluster_labels": list_cluster_labels,
        "cluster_sizes": list_cluster_sizes,
        "min_cluster_size": list_min_cluster_size,
        "silh_samples": list_silh_samples,
        "asw": list_asw,
        "ub_asw": list_ub_asw,
        "ub_asw_min_cluster_size": list_ub_asw_min_cluster_size,
    }
    
    # Save to pickle 
    pd.DataFrame.from_dict(results).to_pickle(path)


In [20]:
# save results 
save_results(path = "results/aloi125-5classes.pkl", diss_matrix=D2, k_range=range(5,6))

100%|██████████| 1/1 [00:00<00:00,  2.34it/s]

0.8571444368288681
